In [12]:
import networkx as nx
import random
import numpy as np

In [13]:
def evol_memory(setnodes,g,passed_contacts,passed_contacts_inf,status):
    
    for i in setnodes:
        if i in g.nodes():
            nc = g.degree(i)        # number of contacts of node i at time t
            nc1 = len([x for x in g.neighbors(i) if status[x]==1])  # number of contacts with nodes of status 1
        else:
            nc = 0
            nc1 = 0
        passed_contacts[i].pop(0)        # removing the oldest memory
        passed_contacts_inf[i].pop(0)
        passed_contacts[i].append(nc)      # adding the data of the current time
        passed_contacts_inf[i].append(nc1)
    return

In [14]:
def cascade(g,passed_contacts,passed_contacts_inf,status,set_inf,phi):

    nodes_to_add = set()

    for node in g.nodes():
        if status[node] == 1:
            continue
        nc = sum(passed_contacts)
        if sum(passed_contacts_inf) > phi*nc:
            nodes_to_add.add(node)
            
    set_inf = set_inf.union(nodes_to_add)
    for node in nodes_to_add:
        status[node] = 1
        
    return

In [23]:
# Temporal network parameters
N = 1000
setnodes = range(N)
Tmax = 10
nruns = 2

# cascade parameters
theta = 10   # memory length
phi = 0.4  # threshold 
initialfraction = 0.01
nseeds = initialfraction*N

average_ninf_AD, average_ninf_SAD = {}, {}
for t in range(Tmax):
    average_ninf_AD[t] = 0
    average_ninf_SAD[t] = 0
    
list_final_sizes_AD, list_final_sizes_SAD = [], [] 

# NOTE: possibility of duplicating for comparing process on AD vs SAD with given seed
#  => passed_contacts_AD = {}, passed_contacts_AD_inf = {}  and passed_contacts_SAD = {}, passed_contacts_SAD_inf = {}


for irun in range(nruns):

    # initialization of seeds: it will be the same set of seeds for both AD and SAD cases
    seeds = set()
    while len(seeds) < nseeds:
        seeds.add(random.choice(setnodes))
        
    for (networktype,average_ninf,list_final_sizes) in zip(['AD','SAD'],
                                                          [average_ninf_AD, average_ninf_SAD],
                                                          [list_final_sizes_AD, list_final_sizes_SAD]):
        passed_contacts = {}
        passed_contacts_inf = {}
        status = {}
        set_inf = set()
        for i in setnodes:
            passed_contacts[i] = [0]*theta
            passed_contacts_inf[i] = [0]*theta
            status[i] = 0

        for seed in seeds:
            status[seed] = 1
            set_inf.add(seed)

        t = 0
        while t < Tmax:
            # step of creation of instantaneous network
            if networktype == 'AD':
                g = nx.Graph()   # creation of AD graph
            else:
                g = nx.Graph()  # creation of SAD graph
            evol_memory(setnodes,g,passed_contacts,passed_contacts_inf,status)
            cascade(g,passed_contacts,passed_contacts_inf,status,set_inf,phi)
          #  print t,len(set_inf)
            average_ninf[t] += len(set_inf)
            t += 1

        list_final_sizes.append(len(set_inf))

for t in range(Tmax):
    print t,average_ninf_AD[t]/float(nruns),average_ninf_SAD[t]/float(nruns)
print np.average(list_final_sizes_AD),np.average(list_final_sizes_SAD)
print list_final_sizes_AD,list_final_sizes_SAD

0 10.0 10.0
1 10.0 10.0
2 10.0 10.0
3 10.0 10.0
4 10.0 10.0
5 10.0 10.0
6 10.0 10.0
7 10.0 10.0
8 10.0 10.0
9 10.0 10.0
10.0 10.0
[10, 10] [10, 10]
